# Monte Carlo simulations of gamma rays observed by LST

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import glob
import h5py
from datetime import datetime
from astropy import units as u
from astropy.coordinates import SkyCoord, EarthLocation, builtin_frames, Angle
from astropy.coordinates.erfa_astrom import ErfaAstromInterpolator, erfa_astrom
from astropy.time import Time
from bokeh.io import output_notebook, output_file
from bokeh.plotting import figure, show
from bokeh.models import ColumnDataSource, Whisker, Step, Title, LinearColorMapper, BasicTicker, ColorBar
from bokeh.layouts import row
from bokeh.palettes import Inferno256, Greys9
import pyirf
from pyirf.spectral import CRAB_MAGIC_JHEAP2015

In [2]:
output_notebook()

Loading BokehJS ...

In [3]:
filename = '/nfs/cta-ifae/moralejo/CTA/LST/MC/DL2/20200629_prod5_trans_80/gamma/zenith_20deg/south_pointing/20210416_v0.7.3_prod5_trans_80_local_taicut_8_4/off0.4deg/dl2_gamma_20deg_180deg_off0.4deg_20210416_v0.7.3_prod5_trans_80_local_taicut_8_4_testing.h5'

MC_data = pd.read_hdf(filename, 'dl2/event/telescope/parameters/LST_LSTCam')

f = h5py.File(filename,'r')
MC_config = f['simulation/run_config']

In [4]:
#energy distribution of the simulated gamma rays:
fig = figure(title = 'Distribution of the energy of the simulated gamma rays', plot_width=450, plot_height=300,
             y_axis_type='log')

n, b = np.histogram(np.log10(MC_data['mc_energy']), bins=100, range = (min(np.log10(MC_data['mc_energy'])),
                                                  max(np.log10(MC_data['mc_energy']))))
fig.quad(top=n, bottom=0.1, left=b[:-1], right=b[1:])

base = (b[1:]+b[:-1])/2
lower = n-np.sqrt(n)/2
upper = n+np.sqrt(n)/2
source_error = ColumnDataSource(data=dict(base=base, lower=lower, upper=upper))
w=Whisker(source=source_error, base='base', upper='upper', lower='lower', line_color='gray')
w.upper_head.line_color = 'gray'
w.lower_head.line_color = 'gray'
fig.add_layout(w)

fig.xaxis.axis_label = 'log(mc_energy)'
fig.yaxis.axis_label = 'counts'
fig.title.text_font_size = '8pt'
fig.title.text_font_style = 'normal'    

show(fig)

In [5]:
def calculate_event_weights(target_spectrum, simulated_spectrum):
    
    return (target_spectrum/simulated_spectrum)

In [6]:
Emin = np.unique(MC_config['energy_range_min'])[0]
Emax = np.unique(MC_config['energy_range_max'])[0]
N,b = np.histogram(MC_data['mc_energy'], bins=100, range=(Emin, Emax))
E = (b[1:]+b[:-1])/2

radius = np.unique(MC_config['max_scatter_range'])[0]*100  #radius of the circle in which the MC is generated (in cm)
area = np.pi*radius**2
simulated_spectrum = N/(E*area)  #dN/(dE dA) en (sucesos/cm²/TeV)

target_spectrum = CRAB_MAGIC_JHEAP2015(E*u.TeV).value

weight = calculate_event_weights(target_spectrum, simulated_spectrum)

In [7]:
fig1 = figure(plot_width=400, plot_height=300, y_axis_type='log', title = 'Target_spectrum (Crab nebula)', 
             x_axis_label='energy (TeV)', y_axis_label='dF/dE (1/TeV/s/cm²)')
fig1.circle(E, target_spectrum)

fig2 = figure(plot_width=400, plot_height=300, y_axis_type='log', title = 'Simulated spectrum',
             x_axis_label='energy (TeV)', y_axis_label='dN/(dE dA) (1/TeV/cm²)')
fig2.circle(E, simulated_spectrum)

show(row(fig1, fig2))

In [8]:
MC_data.shape

(1014253, 60)

In [9]:
area*(np.sum(simulated_spectrum*E))

1014253.0

In [10]:
tot_events = np.sum(MC_config['num_showers']*MC_config['shower_reuse'])   #total number of simulated events
tot_events

50000000

In [12]:
def distribution_gammas(df, par, bins, weight, xlims=None):  #distribution of the given parameter of the simulated gamma rays
    fig = figure(title = 'Distribution of {} of the simulated gamma rays'.format(par),
                  plot_width=450, plot_height=300, x_range=xlims)
    
    n, b = np.histogram(df[par], bins=bins, range = (xlims), weights=weight)  
    fig.quad(top=n, bottom=0, left=b[:-1], right=b[1:])

    base = (b[1:]+b[:-1])/2
    lower = n-np.sqrt(n)/2
    upper = n+np.sqrt(n)/2
    source_error = ColumnDataSource(data=dict(base=base, lower=lower, upper=upper))
    w=Whisker(source=source_error, base='base', upper='upper', lower='lower', line_color='gray')
    w.upper_head.line_color = 'gray'
    w.lower_head.line_color = 'gray'
    fig.add_layout(w)

    fig.xaxis.axis_label = par
    fig.yaxis.axis_label = 'counts'
    fig.title.text_font_size = '8pt'
    fig.title.text_font_style = 'normal'    
    
    show(fig)

In [ ]:
distribution_gammas(MC_data, 'width', 50)

In [ ]:
distribution_gammas(MC_data, 'length', 50)

In [ ]:
distribution_gammas(MC_data, 'log_intensity', 40)